In [1]:
# Loan analysis of Lending Club data for the years 2014,2015,2016

# Load dataset

import pandas as pd

data = pd.read_csv('acceptedloans.csv', nrows=2500)

In [4]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 151 entries, id to settlement_term
dtypes: float64(115), int64(1), object(35)
memory usage: 6.9 MB


In [12]:
pd.options.display.max_columns = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 4000

In [32]:
# Identify float columns, how many have no missing values and could be converted to int

numeric_cols = data.select_dtypes(include=['float'])

floats = []
ints = []

for col in numeric_cols.columns:
    
    if data[col].isnull().values.any() == True:
        floats.append(col)
    elif data[col].isnull().values.any() == False:
        ints.append(col)

In [36]:
for col in floats:
    
    data[col] = pd.to_numeric(data[col], downcast='float')
    
float_cols = data.select_dtypes(include=['float'])
print(float_cols.dtypes)

loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
int_rate                      float64
installment                   float64
annual_inc                    float64
dti                           float64
delinq_2yrs                   float64
fico_range_low                float64
fico_range_high               float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_amnt               float64
last_fico_range_high          float64
last_fico_ra

In [38]:
for col in ints:
    
    data[col] = pd.to_numeric(data[col], downcast='integer')
    
int_cols = data.select_dtypes(include=['integer'])
print(int_cols.dtypes)

id                            int64
loan_amnt                     int32
funded_amnt                   int32
funded_amnt_inv               int32
delinq_2yrs                    int8
fico_range_low                int16
fico_range_high               int16
inq_last_6mths                 int8
open_acc                       int8
pub_rec                        int8
revol_bal                     int32
total_acc                      int8
last_fico_range_high          int16
last_fico_range_low           int16
collections_12_mths_ex_med     int8
policy_code                    int8
acc_now_delinq                 int8
tot_coll_amt                  int32
tot_cur_bal                   int32
open_acc_6m                    int8
open_act_il                    int8
open_il_12m                    int8
open_il_24m                    int8
total_bal_il                  int32
open_rv_12m                    int8
open_rv_24m                    int8
max_bal_bc                    int32
all_util                    

In [39]:
print(data.info(memory_usage='deep'))
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 151 entries, id to settlement_term
dtypes: category(33), float32(42), float64(15), int16(8), int32(15), int64(1), int8(35), object(2)
memory usage: 1.6 MB
None


Further reduction in memory footprint to 1.6MB by converting float64 columns with no missing values to their most appropriate int types

In [26]:
# Identify string columns that could be converted to type category

obj_cols = data.select_dtypes(include=['object'])

for obj in obj_cols.columns:
    
    unique_vals = len(data[obj].unique())
    total_vals = len(data[obj])
    
    if unique_vals / total_vals < 0.5:
        data[obj] = data[obj].astype('category')
    
print(data.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 151 entries, id to settlement_term
dtypes: category(33), float64(115), int64(1), object(2)
memory usage: 2.9 MB
None


Converting string columns to type category reduces memory footprint per chunk from 6.9MB to 2.9MB

In [41]:
### Write a function that automates this process
# Determine optimal chunk size based on memory constraints
# Determine which string columns can be converted to numeric by removing the % character
# Determine which numeric columns can be converted to more space efficient representations
# Convert columns to datetime

data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,93999587,NaN,19200,19200,19200,36 months,13.99,656.12,C,C3,Foreman,10+ years,MORTGAGE,75000.0,Not Verified,Dec-2016,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,home_improvement,Home improvement,531xx,WI,16.98,0,Dec-2003,740,744,0,NaN,NaN,8,0,12802,59.799999,12,w,7057.74,7057.74,15909.570000,15909.57,12142.26,3767.31,0.0,0.0,0.0,Jan-2019,656.12,Feb-2019,Jan-2019,739,735,0,NaN,1,Individual,NaN,NaN,NaN,0,0,243746,0,2,0,2,15.0,6341,39.0,0,0,4126,57,21400,0,0,0,3,30468,7136.0,53.400002,0,0,111.0,80,26,15,3,47.0,NaN,15.0,NaN,0,2,3,2,2,3,3,4,3,8,0.0,0,0,0,100.0,0.000000,0,0,277588,24788,15300,16273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,96219354,NaN,7000,7000,7000,36 months,5.32,210.81,A,A1,Nurse,2 years,RENT,85000.0,Not Verified,Dec-2016,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,112xx,NY,12.80,1,Feb-1992,705,709,0,20.0,30.0,14,1,6565,19.700001,17,w,2458.06,2458.06,5054.270000,5054.27,4541.94,512.33,0.0,0.0,0.0,Jan-2019,210.81,Feb-2019,Jan-2019,709,705,0,NaN,1,Individual,NaN,NaN,NaN,0,0,42606,0,2,0,0,30.0,36041,90.0,2,4,1157,58,33300,0,0,0,4,3043,15577.0,16.700001,0,0,70.0,298,9,9,0,9.0,20.0,18.0,20.0,0,4,7,6,7,2,12,15,7,14,0.0,0,0,2,88.2,0.000000,0,0,73420,42606,18700,40120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,96309331,NaN,12000,12000,12000,36 months,16.99,427.78,D,D1,Na